In [5]:
!pip install folium python-dotenv


[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip


In [6]:
import os
import requests
from dotenv import load_dotenv

In [7]:
load_dotenv()
API_KEY = os.getenv("KAKAO_API_KEY")

In [8]:
def search_places(keyword, page=1, size=15):
    url = "https://dapi.kakao.com/v2/local/search/keyword.json"
    headers = {"Authorization": f"KakaoAK {API_KEY}"}
    params = {"query": keyword, "page": page, "size": size}
    resp = requests.get(url, headers=headers, params=params)
    resp.raise_for_status()
    return resp.json()

In [9]:
keyword = input()

 서경대 맛집


In [10]:
data = search_places(keyword, page=1, size=10)

In [11]:
print("meta:", data.get("meta"))

meta: {'is_end': False, 'pageable_count': 45, 'same_name': {'keyword': '맛집', 'region': [], 'selected_region': '서울 성북구 서경대'}, 'total_count': 5431}


In [12]:
for doc in data.get("documents", []):
    print(doc["place_name"], doc["address_name"], doc["phone"])

청년밥상문간 서울 성북구 정릉동 398-8 010-2808-6031
동방손칼국수 서울 성북구 정릉동 1013 02-909-5953
지하서재 서울 성북구 정릉동 266-328 02-941-1834
마몽함박 서울 성북구 정릉동 289-50 02-6083-2188
기차순대국 서울 성북구 정릉동 347-2 02-914-9316
소우락정육식당 정릉본점 서울 성북구 정릉동 284-13 02-911-9500
홈타운바베큐치킨 서울 성북구 정릉동 227 02-914-0011
임금님수타자장면 서울 성북구 정릉동 298-2 02-914-1831
청수장 서울 성북구 정릉동 416-27 02-913-6176
명륜진사갈비 서울미아점 서울 강북구 미아동 811-1 02-981-6669


In [13]:
import pandas as pd

In [14]:
def docs_to_df(documents):
    rows = []
    for d in documents:
        rows.append({
            "id":d.get("id"),
            "place_name":d.get("place_name"),
            "category_name":d.get("category_name"),
            "phone":d.get("phone"),
            "address":d.get("address_name"),
            "road_address":d.get("road_address_name"),
            "x": float(d.get("x")) if d.get("x") else None,
            "y": float(d.get("x")) if d.get("y") else None,
            "place_url":d.get("palce_url"),
        })
    return pd.DataFrame(rows)

In [15]:
# keyword = input()

In [16]:
dfs = []
page = 1

while True:
    res = search_places(keyword, page=page, size=10)
    df = docs_to_df(res["documents"])
    dfs.append(df)

    if res["meta"]["is_end"]: # 마지막 페이지면 중단
        break

    page += 1 # 다음 페이지로

In [17]:
result = pd.concat(dfs, ignore_index=True)

In [18]:
print(result.shape)
result.head()

(45, 9)


,id,place_name,category_name,phone,address,road_address,x,y,place_url
0,1949561591,청년밥상문간,"음식점 > 한식 > 찌개,전골",010-2808-6031,서울 성북구 정릉동 398-8,서울 성북구 보국문로11길 18-2,127.009121,127.009121,None
1,16819186,동방손칼국수,음식점 > 한식 > 국수 > 칼국수,02-909-5953,서울 성북구 정릉동 1013,서울 성북구 서경로 91,127.013686,127.013686,None
2,2118043633,지하서재,음식점 > 술집,02-941-1834,서울 성북구 정릉동 266-328,서울 성북구 보국문로18길 34,127.010510,127.010510,None
3,1536393758,마몽함박,"음식점 > 양식 > 스테이크,립",02-6083-2188,서울 성북구 정릉동 289-50,서울 성북구 보국문로 92,127.008257,127.008257,None
4,16501643,기차순대국,음식점 > 한식 > 순대,02-914-9316,서울 성북구 정릉동 347-2,서울 성북구 솔샘로18길 91,127.008713,127.008713,None


In [19]:
result.to_csv(f"{keyword}_places.csv", index=False, encoding="utf-8-sig")

In [20]:
!pip install folium


[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip


In [21]:
import folium

In [22]:
df = pd.read_csv(f"{keyword}_places.csv")
center = [37.61498, 127.0134] # 서경대 위도
m = folium.Map(location=center, zoom_start=16)

In [23]:
for _, row in df.dropna(subset=["y","x"]).iterrows():
    folium.Maker(
        [row["y"], row["x"]],
        popup=f"{row['place_name']}<br>{row['category_name']}",
        tooltip=row["place_name"]
    ).add_to(m)

AttributeError: module 'folium' has no attribute 'Maker'

In [24]:
fname = f"{keyword}_placesmap.html"
m.save(fname)
print(f"Saved: {keyword}_places_map.html")

Saved: 서경대 맛집_places_map.html


In [25]:
import webbrowser

In [26]:
path = os.path.abspath(fname)
url = "file://" + path
webbrowser.open(url, new=2)

0:104: execution error: 일부 대상체 파일을 발견할 수 없습니다. (-43)


True